In [1]:
import sys
sys.path.append('..')
import processing

loader = processing.DatasetLoader('..\\..\\dataset\\np_dataset')
# train_dataloader, test_dataloader = loader.get_dataloaders()
random_sample = loader.get_random_sample('train')
src, trg, meta = random_sample

In [2]:
from train import new_model
import torch
type = 'mamba'
model_name = 'loss_3.29_time_2025-03-13-10-43-27.pth'
model = new_model(type)
model.load_state_dict(torch.load(f'../pretrained/{type}/{model_name}'))
model.to('cuda');

C:\Users\Draco\AppData\Local\Temp\ipykernel_10852\2857033724.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'../pretrained/{type}/{mod

In [5]:
output = model(src.unsqueeze(0))
# output.argmax(-1)
output.argmax(-1)

tensor([[ 60, 195, 257, 357, 457, 585,  47, 195, 257, 357, 482, 585,  62, 195,
         257,   0, 482, 585,  47, 195, 257,   0, 482, 585,  68, 195, 257,   0,
         486,   0,  68, 182, 257,   0, 486,   0,  32, 708,   0,   0, 457,   0,
          32, 216,   0,   0, 457,   0,  68, 708,   0,   0, 484,   0,  32, 364,
           0,   0, 484,   0,  32, 364,   0,   0, 457,   0,  68, 708,   0,   0,
         457,   0,  32, 708,   0,   0, 457,   0,  32, 195,   0,   0, 457,   0,
          32, 708,   0,   0, 457,   0,  32, 182,   0,   0, 457,   0,  32, 195,
           0,   0, 457,   0,  32, 195,   0,   0, 457,   0,  68, 194,   0,   0,
         457,   0,  32, 194,   0,   0, 482,   0, 293, 195,   0,   0, 482,   0,
         723, 183]], device='cuda:0')